# Louvain Community Detection


In this notebook, we will use cuGraph to identify the cluster in a test graph using the Louvain algorithm  

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Created:   08/01/2019
* Last Edit: 05/27/2020

RAPIDS Versions: 0.14

Test Hardware
* GV100 32G, CUDA 10.2



## Introduction

The Louvain method of community detection is a greedy hierarchical clustering algorithm which seeks to optimize Modularity as it progresses. Louvain starts with each vertex in its own clusters and iteratively merges groups using graph contraction.  

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Louvain_Modularity

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score

To compute the Louvain cluster in cuGraph use: <br>
   __df, mod  = cugraph.louvain(G, max_iter = 100)__
   
  
    Parameters
    ----------
    input_graph : cugraph.Graph
        cuGraph graph descriptor, should contain the connectivity information
        as an edge list. The adjacency list will be computed if not already present.
        The graph should be undirected where an undirected edge is represented by a
        directed edge in both direction.

    max_iter : integer
        This controls the maximum number of levels/iterations of the Louvain
        algorithm. When specified the algorithm will terminate after no more
        than the specified number of iterations. No error occurs when the
        algorithm terminates early in this manner.

    Returns
    -------
    parts : cudf.DataFrame
        A GPU data frame of size V containing two columns the vertex id and the
        partition id it is assigned to.
        
            df[‘vertex’] cudf.Series
                Contains the vertex identifiers
            df[‘partition’] cudf.Series
                Contains the partition assigned to the vertices
        
        
    modularity_score : float
        a floating point number containing the modularity score of the
        partitioning.
        
    All vertices with the same partition ID are in the same cluster
        


#### Note
Parallel Louvain produces different modularity scores that serial Louvain.

Serial Louvain is a greedy method that iterates over the vertices in numerical order and
evaluates whether to move a vertex into a different cluster and then moves it if
modularity is increased.  After considering all vertices in serial it computes a new
modularity score and if better than the last clustering it will repeat.  If vertex i is
moved to a new cluster, and vertex j is a neighbor of vertex i and is processed after
vertex i, then vertex j will compute its delta modularity based on the vertex i's new
cluster assignment.

Parallel Louvain computes delta modularity for all vertex/cluster combinations and chooses
the best change for ALL vertices in parallel.  This means that vertex i and vertex j being
neighbors (as above), both will see the old cluster assignment for that vertex.  This will result
in different decisions being made between the parallel and serial versions.  Because Louvain
is a greedy method, those decisions might lead the algorithms to converge on different local
maximums.

A complete technical write-up is being produced and will be linked here when available.  

### References
* Blondel, V. D., Guillaume, J.-L., Lambiotte, R., and Lefebvre, E. Fast unfolding of communities in large networks. Journal of statistical mechanics: theory and experiment 2008, 10 (2008), P10008.


## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

The test data has vertex IDs strating at 1.  We will be using the auto-renumber feature of cuGraph to renumber the data so that the starting vertex ID is zero.  The data will be auto-unrenumbered so that the renumbering step is transparent to users. 


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf

## Read data using cuDF

In [ ]:
# Test file    
datafile='../data//karate-data.csv'

In [ ]:
# read the data using cuDF
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

In [ ]:
# The algorithm also requires that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

In [ ]:
# just for fun, let's look at the data types in the dataframe
gdf.dtypes

In [ ]:
# create a Graph - since the data does not start at '0', use the auto-renumbering feature
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='data', renumber=True)

In [ ]:
# Call Louvain on the graph
df, mod = cugraph.louvain(G) 

In [ ]:
# Print the modularity score
print('Modularity was {}'.format(mod))
print()

In [ ]:
df.dtypes

In [ ]:
# How many partitions where found
part_ids = df["partition"].unique()

In [ ]:
print(str(len(part_ids)) + " partition detected")

In [ ]:
# print the clusters.  
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'].iloc[i] == p):
            part.append(df['vertex'].iloc[i] )
    print("Partition " + str(p) + ":")
    print(part)


___
Copyright (c) 2019-2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___